**Laboratory Lecture 6**

## EXAMPLE: Sampling from a probability distribution

All the algorithms for approximate inference with Bayesian Networks require a method for sampling from a known probability distribution.

Let's see a possible implementation of such sampling method in case of Boolean variables and known CPTs:

In [1]:
import numpy as np
import random as rnd

t, f = 0, 1

def samplegen(Pdist, Parents = []):
	assert len(Parents) < len(Pdist.shape)
	if rnd.random() < Pdist[t][tuple(Parents)]:
		return t
	return f

The input of this funtion is a probability distribution **Pdist** with the content of the CPT and, if available, the values of the **Parents** events.

Let's generate some samples from the Sprinkler network:

<img src='https://drive.google.com/uc?id=1xV8HNKqH1_IUF3LWK6VzeMHK1z1VhOfv'>

In particular, let's say we want to generate 20 samples from the $Cloudy$ distribution, ${\bf P}(C)$, which has no parents:

In [3]:
P_C = np.array([0.5, 0.5])

for i in range(20):
    s = samplegen(P_C)
    print("C = ", 'TRUE' if s == t else 'FALSE')

C =  TRUE
C =  FALSE
C =  TRUE
C =  FALSE
C =  FALSE
C =  FALSE
C =  TRUE
C =  FALSE
C =  FALSE
C =  FALSE
C =  FALSE
C =  FALSE
C =  FALSE
C =  FALSE
C =  TRUE
C =  TRUE
C =  FALSE
C =  TRUE
C =  FALSE
C =  FALSE


As expected, there are approximately 50% $true$ and 50% $false$ outcomes.

Let's sample from a conditional distribution, for example ${\bf P}(R | \neg c)$:

In [7]:
P_R_C = np.array([[0.8, 0.1],[0.2, 0.9]])

for i in range(20):
    s = samplegen(P_R_C, [f])
    print("R = ", 'TRUE' if s == t else 'FALSE')

R =  TRUE
R =  FALSE
R =  FALSE
R =  FALSE
R =  TRUE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  TRUE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  TRUE
R =  FALSE


In this case there are many more $false$ than $true$ outcomes because ${\bf P}(R | \neg c) = \langle 0.1, 0.9\rangle$.

Finally, let's sample ${\bf P}(W | \neg s, r) = \langle 0.9, 0.1\rangle$:

In [8]:
P_W_SR = np.array([[[0.95, 0.9],[0.9, 0.1]],[[0.05, 0.1],[0.1, 0.9]]])

for i in range(20):
    s = samplegen(P_W_SR, [f, t])
    print("W = ", 'TRUE' if s == t else 'FALSE')

W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  FALSE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE
W =  TRUE


Even in this case, the samples are more or less as expected, about 90% $true$ and 10% $false$.

## EXAMPLE: Prior-Sampling

Let's implement now the *Prior Sampling* algorithm (see the lecture slides of *Bayesian Networks II* or Chapter 14 of Russell & Norvig's book), starting with some data structures to represent the Sprinkler network:

In [9]:
# some of these distributions were already defined before, but we repeate them just in case
P_C = np.array([0.5, 0.5])
P_S_C = np.array([[0.1, 0.5],[0.9, 0.5]])
P_R_C = np.array([[0.8, 0.1],[0.2, 0.9]])
P_W_SR = np.array([[[0.95, 0.9],[0.9, 0.1]],[[0.05, 0.1],[0.1, 0.9]]])

# network variables...
var = ['C','S','R','W']
# their distributions...
prd = {'C':P_C, 'S':P_S_C, 'R':P_R_C, 'W':P_W_SR}
# their parents...
par = {'C':[], 'S':['C'], 'R':['C'], 'W':['S','R']}
# and their initial values
val = {'C':f, 'S':f, 'R':f, 'W':f}


Let's define also a function to retrieve the values of the parents of a variable:

In [10]:
def parents(X):
	return [val[i] for i in par[X]]

The following algorithm generates 1000 events from the Sprinkler network:

In [24]:
event = []

for n in range(1000):
	for x in var:
		val[x] = samplegen(prd[x], parents(x))
	event.append(['f' if val[x] else 't' for x in var])

print("First randomly generated event = ", event[0])
print("Number or randomly generated events = ", len(event))

P_query = event.count(['t', 'f', 't', 't']) / len(event)
print("P(c,¬s,r,w) = ", P_query)

First randomly generated event =  ['t', 'f', 't', 't']
Number or randomly generated events =  1000
P(c,¬s,r,w) =  0.324


Finally, we can compute the probability of any event by counting the number of times it was generated and normalising. For example, we can verify that $P(c, \neg s, r, w) = 0.328 \approx \dfrac{N_{PS}(c, \neg s, r, w)}{N} = {\hat P}(c, \neg s, r, w)$, as predicted in the lecture:

which is indeed very close to the exact probability!


---

## EXAMPLE 1: Excel for Filtering (Day 1)

The spreadsheet <a href="https://docs.google.com/spreadsheets/d/1BeARmsvIf-pHuuPgICcgyTTOqZq8yC9-/edit?usp=share_link&ouid=116452643742209360003&rtpof=true&sd=true">Umbrella.xlsx</a> models the umbrella example over the first 2 days.


<img src='https://drive.google.com/uc?id=1QkA4DpBqRRl5bc9opUvqm16S30_a-Npd'>


On the top line, the probability of rain for Day 0 is the prior probability.

At the bottom of the sheet are the conditional probability tables for the transition model and the sensor model.

The predicted probability for rain on Day 1 (top) is computed from the probability for Day 0 and the transition model.

To get the filtered probability, we have to bring in information about whether we saw an umbrella or not. The filtered probability of rain for Day 1 (middle of the sheet) is computed by combining the predicted probability for
Day 1, the sensor model, and what we know about umbrellas.

<b>Note:</b> There are two versions of the filtered probability. The raw values which we get directly from the
calculation, and the normalized values (raw values sclaed so they add to 1).

Look at what happens if you change the probability of <tt>umbrella/not umbrella</tt>. Currently the values say
you see an umbrella (probability of <tt>umbrella</tt> is 1 and that of <tt>not umbrella</tt> is 0).
What happens if you don't see an umbrella (probability of <tt>umbrella</tt> is 0 and that of <tt>not umbrella</tt> is 1)?

What about if you have no information (probability of <tt>umbrella</tt> is 0.5 and that of <tt>not umbrella</tt> is 0.5)?


The column for Day 2 just repeats the calculations for Day 1, but starting from the results from Day 1.

Thus the predicted probability for Day 2 is calculated by applying the transition model to the (normalized) filtered probability for Day 1.

The filtered probability of Day 2 is calculated from the predicted probability for Day 2, the sensor model, and what we know about umbrellas.

In other words, the probabilities for Day 2 are computed just like those for Day 1. The calculation is modular.

Look at what happens when the probabilities of umbrella/not umbrella on Days 1 and 2 vary

Pomegranate for Predicting and Filtering

For this example we will use a Python package called <tt>pomegranate</tt>, which provides support for probabilistic reasoning. If you have not installed it before, you will need to do so with:

<tt>pip install pomegranate</tt>

Then you can run the following version of the umbrella model. <tt>pomegranate</tt> can only solve Bayesian neworks (not Dynamic Bayesian Networks), so we have to unroll the whole example to the depth that we want.

In [3]:
#!pip install numpy==1.23.0 --ignore-installed
#!pip install pomegranate==0.14.6
import pomegranate

# Variables are RainN and UmbrellaN+1 for N = 0, 1, ...
# We have a prior for Rain0, two values 'y'es and 'n'o:
Rain0   = DiscreteDistribution({'y': 0.5, 'n': 0.5})

# Transition model
#
# Conditional distribution relating RainN and RainN+1. Notation for
# the conditional probability table is:
#
# [ 'RainN', 'RainN+1', <probability>]
#
# for the conditional value P(Sprinkler|Cloudy). Note that we have to
# repeat the transition model for each pair of states
Rain1 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain0])

Rain2 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain1])

# Sensor model
#
# Conditional distribution relating Rain and Umbrella:
#
# [ 'Umbrella', 'Rain', <probability>]
#
# for the conditional value P(Sprinkler|Cloudy). Values for Umbrella are 'y'es and 'n'o.
# Again we have to enter the table for each day.
Umbrella1 = ConditionalProbabilityTable(
        [['y', 'y', 0.9],
         ['y', 'n', 0.1],
         ['n', 'y', 0.2],
         ['n', 'n', 0.8]], [Rain1])

Umbrella2 = ConditionalProbabilityTable(
        [['y', 'y', 0.9],
         ['y', 'n', 0.1],
         ['n', 'y', 0.2],
         ['n', 'n', 0.8]], [Rain2])
#
# The whole network has five nodes:
s1 = Node(Rain0, name="Rain0")
s2 = Node(Rain1, name="Rain1")
s3 = Node(Umbrella1, name="Umbrella1")
s4 = Node(Rain2, name="Rain2")
s5 = Node(Umbrella2, name="Umbrella2")
# Create a network that includes nodes and edges between them:
model = BayesianNetwork("Umbrella Network")
model.add_states(s1, s2, s3, s4, s5)
model.add_edge(s1, s2)
model.add_edge(s2, s3)
model.add_edge(s2, s4)
model.add_edge(s4, s5)
# Fix the model structure
model.bake()

NameError: name 'DiscreteDistribution' is not defined

Now that we have the model entered, we can ask it questions. We can first ask it to predict the probability of rain on days 1 and 2:

In [ ]:
# Do not instantiate any of the variables:
scenario = [[None, None, None, None, None]]
# Run the model
results =  model.predict_proba(scenario)
# Ask for the probability of rain on Day 1:
print(results[0][1].items())
# And the probability of rain on Day 2:
print(results[0][3].items())

(('n', 0.4999999999999999), ('y', 0.5000000000000001))
(('n', 0.4999999999999999), ('y', 0.5000000000000001))


So both Day 1 and Day 2 have a probability 0.5 of being rainy before we see any umbrellas.

In Bayesian probability terms, this tells us that we can't say anything about how likely it is to rain
(a binary variable with probability of 0.5 for both values is how we represent ¯\_(ツ)_/¯ ).



The reason that we ask for elements 1 and 3 of the datastructure <tt>results</tt> is because they are
elements 1 and 3 of
<tt>model.add_states(s1, s2, s3, s4, s5)</tt>

Now let's tell the model that we see an umbrella on Day 1 and see what that gets us:

In [ ]:
# Set Umbrella on Rain1 to 'y'
scenario = [[None, None, 'y', None, None]]
# Run the model
results =  model.predict_proba(scenario)
# Ask for the probability of rain on Day 1:
print(results[0][1].items())
# And the probability of rain on Day 2:
print(results[0][3].items())

(('n', 0.1818181818181821), ('y', 0.8181818181818179))
(('n', 0.3727272727272729), ('y', 0.6272727272727271))


So it has filtered the probability of rain for Day 1, and also predicted the probability for Day 2 as well.

That is because pomegranate propagates all updates through the whole model/network. It has, for example also computed the probability of rain on Day 0 (that it rained on Day 0 even though we said noting about the rain that day):

In [ ]:
# Ask for the probability of rain on Day 0:
print(results[0][0].items())

(('y', 0.6272727272727271), ('n', 0.372727272727273))


This is what we call the smoothed probability of rain on Day 0.

Now let's tell <tt>pomegranate</tt> about rain on Day 3, so we need to add information about Day3:

In [ ]:
# Conditional probability table
Rain3 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain2])
# Node
s6 = Node(Rain3, name="Rain3")
# State
model.add_states(s6)
# Edge
model.add_edge(s4, s6)
# Fix the model structure
model.bake()


Note that we only call model.bake() once the last elements are entered.

Now that we have the model entered, we can ask it questions. We tell the model that we saw Umbrellas on Days 1 and 2:

In [ ]:
# Umbrellas on Day 1 and 2:
scenario = [[None, None, 'y', None, 'y', None]]
# Run the model
results =  model.predict_proba(scenario)
# Ask for the probability of rain on Day 1:
print(results[0][1].items())
# And the probability of rain on Day 2:
print(results[0][3].items())

(('n', 0.11664295874822224), ('y', 0.8833570412517778))
(('n', 0.11664295874822232), ('y', 0.8833570412517777))


Note that we didn't tell pomegranate to do smoothing. As we saw before with Day 0, it (in effect) always runs the backwards propagation and gives us smoothed probabilities for all days before the latest piece of evidence.

I said "in effect" because pomegranate doesn't do the computation the way we studied. It just computes the probability of every hidden variable given the evidence.